# **CONJUNTO DE DATOS**

El conjunto de datos utilizado en este modelo está formado por datos de libros de publicados sobre varios países de todo el mundo. Cubre una amplia gama de temas que incluyen indicadores económicos, estadísticas demográficas, detalles geográficos, factores sociales y más. Este conjunto de datos completo proporciona información valiosa sobre diferentes aspectos de los países, lo que permite realizar análisis y comparaciones en profundidad.

Para nuestro ejemplo, nos centraremos específicamente en un subconjunto de datos de Factbook relacionados con factores económicos como datos de importaciones y exportaciones, PIB (Producto Interno Bruto), inversión, tasa de desempleo y tasa de crecimiento de la producción industrial. Estos indicadores económicos desempeñan un papel crucial en la comprensión del desempeño económico, las tendencias y las áreas potenciales de crecimiento o preocupación de un país.

*   **Datos independientes: exportaciones, importaciones, inversión, tasa de desempleo, tasa de crecimiento de la producción industrial.**
*   **Datos dependientes : PIB**

# **MODELO**

El modelo utilizado en este código es un perceptrón multicapa (MLP) implementado con PyTorch. El MLP consta de tres capas completamente conectadas. La capa de entrada tiene 5 neuronas, que representan las 5 características de entrada. Las capas ocultas tienen 64 y 32 neuronas respectivamente, y las funciones de activación ReLU se aplican después de cada capa oculta. La capa de salida tiene 1 neurona, que proporciona la salida prevista.

El método directo define el paso directo del MLP, donde los datos de entrada se propagan a través de las capas para generar la salida. Las capas se definen utilizando el módulo nn.Sequential, que permite una disposición secuencial de las capas.

Durante el entrenamiento, la función de pérdida L1 (nn.L1Loss) se utiliza para calcular la pérdida entre las salidas previstas y los valores objetivo. El optimizador Adagrad (torch.optim.Adagrad) se utiliza con una tasa de aprendizaje de 1e-4 para actualizar los parámetros del modelo y minimizar la pérdida. El proceso de entrenamiento se realiza durante 5 epochs, y cada epoch consta de iteraciones sobre los datos de entrenamiento.

La pérdida después de cada minilote se imprime para monitorear el progreso del entrenamiento. Al final de cada epoch, se restablece la pérdida actual. Finalmente se indica la finalización del proceso formativo.

En general, la arquitectura de este modelo, la función de pérdida y la configuración del optimizador tienen como objetivo entrenar el MLP para predecir la variable objetivo en función de las características de entrada y minimizar la pérdida L1 durante el proceso de entrenamiento.

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn
from torch.utils.data import DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
df = pd.read_csv('clean.csv')
df.head()

,exports,imports,industrial production growth rate,investment,unemployement rate,gdp
0,0.126491,0.024505,0.280423,0.161121,0.351585,0.026382
1,0.181904,0.062921,0.375661,0.194396,0.110951,0.119814
2,0.146305,0.082021,0.330688,0.217163,0.040346,0.021420
3,0.085787,0.031111,0.343915,0.166375,0.053314,0.015557
4,0.353292,0.173487,0.259259,0.199650,0.092219,0.087047


In [ ]:
X = df.drop('gdp', axis=1).values
y = df['gdp'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
class factbook_data:
    def __init__(self, X, y, scale_data=True):
        if not torch.is_tensor(X) and not torch.is_tensor(y):
            if scale_data:
                X = StandardScaler().fit_transform(X)
        self.X = torch.from_numpy(X)
        self.y = torch.from_numpy(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [ ]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(5, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.layers(x)

In [ ]:
if __name__=='__main__':
    torch.manual_seed(42)
    X,y = X, y


In [ ]:
dataset = factbook_data(X, y, scale_data=False)
trainloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)
testloader = torch.utils.data.DataLoader(dataset, batch_size=10, shuffle=True, num_workers=1)

In [ ]:
mlp = MLP()

loss_function = nn.L1Loss()
optimizer = torch.optim.Adagrad(mlp.parameters(), lr=1e-4)

In [ ]:
for epoch in range(0,5):
    print(f'Starting Epoch {epoch+1}')

    current_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        inputs, targets = data
        inputs, targets = inputs.float(), targets.float()
        targets = targets.reshape((targets.shape[0], 1))

        optimizer.zero_grad()

        outputs = mlp(inputs)

        loss = loss_function(outputs, targets)

        loss.backward()

        optimizer.step()

        current_loss += loss.item()

        if i%10 == 0:
            print(f'Loss after mini-batch %5d: %.3f'%(i+1, current_loss/500))
            current_loss = 0.0

    print(f'Epoch {epoch+1} finished')

print("Training has completed")

Starting Epoch 1
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.001
Epoch 1 finished
Starting Epoch 2
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.001
Epoch 2 finished
Starting Epoch 3
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.001
Epoch 3 finished
Starting Epoch 4
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.001
Epoch 4 finished
Starting Epoch 5
Loss after mini-batch     1: 0.000
Loss after mini-batch    11: 0.001
Epoch 5 finished
Training has completed


In [ ]:
test_data = torch.from_numpy(X_test).float()
test_targets = torch.from_numpy(y_test).float()

In [ ]:
mlp.eval()

MLP(
  (layers): Sequential(
    (0): Linear(in_features=5, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=1, bias=True)
  )
)

In [ ]:
with torch.no_grad():
    outputs = mlp(test_data)
    predicted_labels = outputs.squeeze().tolist()

predicted_labels = np.array(predicted_labels)
test_targets = np.array(test_targets)

mse = mean_squared_error(test_targets, predicted_labels)
r2 = r2_score(test_targets, predicted_labels)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.0036111390217584464
